In [223]:
import pandas as pd
import numpy as np

In [224]:
gdp = '/Users/nidhipareddy/Desktop/uchicago/Quilo/GDP.xlsx'
unemployment_rt = '/Users/nidhipareddy/Desktop/uchicago/Quilo/UNRATE.xlsx'
cpi = '/Users/nidhipareddy/Desktop/uchicago/Quilo/CPIAUCSL.xlsx'
treasuty_yield_10Y = '/Users/nidhipareddy/Desktop/uchicago/Quilo/GS10.xlsx'
sp500 = '/Users/nidhipareddy/Desktop/uchicago/Quilo/VXVCLS.xlsx'
crepi = '/Users/nidhipareddy/Desktop/uchicago/Quilo/COMREPUSQ159N.xlsx'
csi = '/Users/nidhipareddy/Desktop/uchicago/Quilo/UMCSENT.xlsx'
delinquency = '/Users/nidhipareddy/Desktop/uchicago/Quilo/DRALACBS.xlsx'
pce = '/Users/nidhipareddy/Desktop/uchicago/Quilo/PCE.xlsx'

In [225]:
# Load the GDP data from the 'Quarterly' sheet
gdp_df = pd.read_excel(gdp, sheet_name="Quarterly")

# Optional: convert date column to datetime
gdp_df['observation_date'] = pd.to_datetime(gdp_df['observation_date'])

# Optional: rename columns for consistency
gdp_df.rename(columns={'observation_date': 'date', 'GDP': 'gdp'}, inplace=True)

In [226]:
unemployment_df = pd.read_excel(unemployment_rt, sheet_name="Monthly")
unemployment_df['observation_date'] = pd.to_datetime(unemployment_df['observation_date'])
unemployment_df.rename(columns={'observation_date': 'date', 'UNRATE': 'unemployment_rate'}, inplace=True)

In [227]:
cpi_df = pd.read_excel(cpi, sheet_name="Monthly")
cpi_df['observation_date'] = pd.to_datetime(cpi_df['observation_date'])
cpi_df.rename(columns={'observation_date': 'date', 'CPIAUCSL': 'cpi'}, inplace=True)

In [228]:
treasury_yield_df = pd.read_excel(treasuty_yield_10Y, sheet_name="Monthly")
treasury_yield_df['observation_date'] = pd.to_datetime(treasury_yield_df['observation_date'])
treasury_yield_df.rename(columns={'observation_date': 'date', 'GS10': 'treasury_yield'}, inplace=True)

In [229]:
sp500_df = pd.read_excel(sp500, sheet_name="Daily, Close")
sp500_df['observation_date'] = pd.to_datetime(sp500_df['observation_date'])
sp500_df.rename(columns={'observation_date': 'date', 'VXVCLS': 'sp500'}, inplace=True)

In [230]:
crepi_df = pd.read_excel(crepi, sheet_name="Quarterly")
crepi_df['observation_date'] = pd.to_datetime(crepi_df['observation_date'])
crepi_df.rename(columns={'observation_date': 'date', 'COMREPUSQ159N': 'crepi'}, inplace=True)

In [231]:
# Ensure date column is datetime and set as index
crepi_df['date'] = pd.to_datetime(crepi_df['date'])
crepi_df.set_index('date', inplace=True)

# Create a full quarterly date range using Quarter Start 
full_range = pd.date_range(start=crepi_df.index.min(), end='2025-03-31', freq='QS')

# Reindex with full date range and forward-fill missing values
crepi_df = crepi_df.reindex(full_range)
crepi_df.ffill(inplace=True)

# Reset index and rename if needed
crepi_df.reset_index(inplace=True)
crepi_df.rename(columns={'index': 'date'}, inplace=True)

print(crepi_df.head())

        date     crepi
0 2011-04-01  1.867975
1 2011-07-01  1.423464
2 2011-10-01  6.368259
3 2012-01-01  2.673520
4 2012-04-01  4.217158


In [232]:
for df in [gdp_df, unemployment_df, cpi_df, treasury_yield_df, sp500_df, crepi_df]:
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'].dt.to_period('M').dt.to_timestamp(how='start')

In [233]:
# Convert date column to datetime
sp500_df['date'] = pd.to_datetime(sp500_df['date'])

# Set date as index
sp500_df.set_index('date', inplace=True)

# Resample by month using mean, then set date to first of month
sp500_monthly = sp500_df.resample('M').mean().reset_index()

# Change dates from end-of-month to first-of-month
sp500_monthly['date'] = sp500_monthly['date'].dt.to_period('M').dt.to_timestamp(how='start')

print(sp500_monthly.head())

        date      sp500
0 2011-03-01  19.840000
1 2011-04-01  19.122000
2 2011-05-01  18.933333
3 2011-06-01  20.403182
4 2011-07-01  20.264000


In [234]:
# Resample by quarter using mean, reset index
sp500_quarterly = sp500_df.resample('Q').mean().reset_index()

# Convert to first day of each quarter
sp500_quarterly['date'] = sp500_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start')

print(sp500_quarterly.head())

        date      sp500
0 2011-01-01  19.840000
1 2011-04-01  19.506508
2 2011-07-01  29.693906
3 2011-10-01  31.800794
4 2012-01-01  21.766935


In [235]:
# Ensure date is datetime and set as index
unemployment_df['date'] = pd.to_datetime(unemployment_df['date'])
unemployment_df.set_index('date', inplace=True)

# Resample by calendar quarter, take mean, then reset index
unemployment_quarterly = unemployment_df.resample('Q').mean().reset_index()

# Change dates to first day of each quarter
unemployment_quarterly['date'] = unemployment_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start')

print(unemployment_quarterly.head())

        date  unemployment_rate
0 2011-04-01           9.066667
1 2011-07-01           9.000000
2 2011-10-01           8.633333
3 2012-01-01           8.266667
4 2012-04-01           8.200000


In [236]:
# Convert date to datetime and set as index
cpi_df['date'] = pd.to_datetime(cpi_df['date'])
cpi_df.set_index('date', inplace=True)

# Resample to quarterly using mean, reset index
cpi_quarterly = cpi_df.resample('Q').mean().reset_index()

# Set the date to first day of each quarter
cpi_quarterly['date'] = cpi_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start')

print(cpi_quarterly.head())

        date         cpi
0 2011-04-01  224.568333
1 2011-07-01  226.032667
2 2011-10-01  227.047333
3 2012-01-01  228.326000
4 2012-04-01  228.808000


In [237]:
# Convert date to datetime and set as index
treasury_yield_df['date'] = pd.to_datetime(treasury_yield_df['date'])
treasury_yield_df.set_index('date', inplace=True)

# Resample to quarterly using mean, reset index
treasury_yield_quarterly = treasury_yield_df.resample('Q').mean().reset_index()

# Set the date to first day of each quarter
treasury_yield_quarterly['date'] = treasury_yield_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start')

print(treasury_yield_quarterly.head())

        date  treasury_yield
0 2011-04-01        3.210000
1 2011-07-01        2.426667
2 2011-10-01        2.046667
3 2012-01-01        2.036667
4 2012-04-01        1.823333


In [238]:
# Start with one base DataFrame
merged_df = gdp_df.copy()

# Merge other dataframes on 'date'
merged_df = merged_df.merge(sp500_quarterly, on='date', how='left')
merged_df = merged_df.merge(unemployment_quarterly, on='date', how='left')
merged_df = merged_df.merge(cpi_quarterly, on='date', how='left')
merged_df = merged_df.merge(treasury_yield_quarterly, on='date', how='left')
merged_df = merged_df.merge(crepi_df, on='date', how='left')

merged_df['date'] = merged_df['date'].dt.date

merged_df.head()

,date,gdp,sp500,unemployment_rate,cpi,treasury_yield,crepi
0,2011-01-01,15351.448,19.840000,NaN,NaN,NaN,NaN
1,2011-04-01,15557.539,19.506508,9.066667,224.568333,3.210000,1.867975
2,2011-07-01,15647.680,29.693906,9.000000,226.032667,2.426667,1.423464
3,2011-10-01,15842.259,31.800794,8.633333,227.047333,2.046667,6.368259
4,2012-01-01,16068.805,21.766935,8.266667,228.326000,2.036667,2.673520


In [239]:
start_date = merged_df['date'].min()
end_date = merged_df['date'].max()

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2011-01-01
End Date: 2025-01-01


In [240]:
csi_df = pd.read_excel(csi, sheet_name="Monthly")
csi_df['observation_date'] = pd.to_datetime(csi_df['observation_date'])
csi_df.rename(columns={'observation_date': 'date', 'UMCSENT': 'consumer_sentiment'}, inplace=True)

In [241]:
# Convert to datetime just in case
csi_df['date'] = pd.to_datetime(csi_df['date'])

# Set date as index
csi_df.set_index('date', inplace=True)

# Resample to quarterly using mean
csi_quarterly = csi_df.resample('Q').mean().reset_index()

# Change the quarterly dates to the *start* of the quarter and remove timestamp
csi_quarterly['date'] = csi_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start').dt.date

print(csi_quarterly.head())

         date  consumer_sentiment
0  2011-01-01           73.066667
1  2011-04-01           71.866667
2  2011-07-01           59.666667
3  2011-10-01           64.800000
4  2012-01-01           75.500000


In [242]:
deli_df = pd.read_excel(delinquency, sheet_name="Quarterly, End of Period")
deli_df['observation_date'] = pd.to_datetime(deli_df['observation_date'])
deli_df.rename(columns={'observation_date': 'date', 'DRALACBS': 'delinquency_rate'}, inplace=True)

In [243]:
deli_df.head()

,date,delinquency_rate
0,2011-01-01,6.20
1,2011-04-01,5.96
2,2011-07-01,5.70
3,2011-10-01,5.46
4,2012-01-01,5.27


In [244]:
pce_df = pd.read_excel(pce, sheet_name="Monthly")
pce_df['observation_date'] = pd.to_datetime(pce_df['observation_date'])
pce_df.rename(columns={'observation_date': 'date', 'PCE': 'pce'}, inplace=True)

In [245]:
# Convert to datetime just in case
pce_df['date'] = pd.to_datetime(pce_df['date'])

# Set date as index
pce_df.set_index('date', inplace=True)

# Resample to quarterly using mean
pce_quarterly = pce_df.resample('Q').mean().reset_index()

# Change the quarterly dates to the *start* of the quarter and remove timestamp
pce_quarterly['date'] = pce_quarterly['date'].dt.to_period('Q').dt.to_timestamp(how='start').dt.date

print(pce_quarterly.head())

         date           pce
0  2011-01-01  10558.200000
1  2011-04-01  10673.033333
2  2011-07-01  10755.033333
3  2011-10-01  10809.166667
4  2012-01-01  10959.300000


In [246]:
# Ensure all date columns are datetime
csi_quarterly['date'] = pd.to_datetime(csi_quarterly['date'])
deli_df['date'] = pd.to_datetime(deli_df['date'])
pce_quarterly['date'] = pd.to_datetime(pce_quarterly['date'])
merged_df['date'] = pd.to_datetime(merged_df['date'])  # This fixes your error

# Proceed with merging
merged_df = merged_df.merge(csi_quarterly, on='date', how='left')
merged_df = merged_df.merge(deli_df, on='date', how='left')
merged_df = merged_df.merge(pce_quarterly, on='date', how='left')


In [247]:
merged_df.head()

,date,gdp,sp500,unemployment_rate,cpi,treasury_yield,crepi,consumer_sentiment,delinquency_rate,pce
0,2011-01-01,15351.448,19.840000,NaN,NaN,NaN,NaN,73.066667,6.20,10558.200000
1,2011-04-01,15557.539,19.506508,9.066667,224.568333,3.210000,1.867975,71.866667,5.96,10673.033333
2,2011-07-01,15647.680,29.693906,9.000000,226.032667,2.426667,1.423464,59.666667,5.70,10755.033333
3,2011-10-01,15842.259,31.800794,8.633333,227.047333,2.046667,6.368259,64.800000,5.46,10809.166667
4,2012-01-01,16068.805,21.766935,8.266667,228.326000,2.036667,2.673520,75.500000,5.27,10959.300000


In [248]:
LEAP_df = pd.read_excel('/Users/nidhipareddy/Desktop/uchicago/Quilo/LEAP_Calculated.xlsx')

In [249]:
LEAP_df.head()

,cycle_date,cu_number,total_assets,total_loans,new_and_used_vehicles,first_and_junior_lien_mtges,commercial_loans,ncuaid,city,state,...,leap_score,county_code,residential_ratio,business_ratio,other_ratio,total_ratio,cbsa_code,msa_name,msa_type,csa_title
0,2025-03-31,1,12576947,9128084,6323733,0,0,1,Texarkana,TX,...,0.692778,48067.0,0.000047,0.000000,0.000000,0.000044,NaN,NaN,NaN,NaN
1,2025-03-31,6,276282959,204137625,102809329,35070948,1529923,6,Metairie,LA,...,0.682923,22051.0,0.005724,0.009012,0.000175,0.005791,35380.0,"New Orleans-Metairie, LA",Metropolitan Statistical Area,"New Orleans-Metairie-Slidell, LA-MS"
2,2025-03-31,12,65700600,31313215,8991363,8516880,0,12,Hartford,CT,...,0.559133,9110.0,0.022268,0.023739,0.048360,0.023909,25540.0,"Hartford-West Hartford-East Hartford, CT",Metropolitan Statistical Area,"New Haven-Hartford-Waterbury, CT"
3,2025-03-31,13,1207819927,1039684783,615757977,378510173,1088680,13,Baton Rouge,LA,...,0.957364,22033.0,0.092280,0.114442,0.142022,0.095945,12940.0,"Baton Rouge, LA",Metropolitan Statistical Area,"Baton Rouge-Hammond, LA"
4,2025-03-31,16,9971825,3289205,1904889,895139,0,16,Omaha,NE,...,0.851278,31055.0,0.027916,0.058048,0.029684,0.030559,36540.0,"Omaha, NE-IA",Metropolitan Statistical Area,"Omaha-Fremont, NE-IA"


In [250]:
# Ensure date is datetime
LEAP_df['cycle_date'] = pd.to_datetime(LEAP_df['cycle_date'])

# Group by date and sum total lending
lending_ts = LEAP_df.groupby('cycle_date')['total_loans'].sum().reset_index()

# Rename for clarity
lending_ts.rename(columns={'cycle_date': 'date', 'total_loans': 'total_lending_volume'}, inplace=True)

In [251]:
# Snap each date to the beginning of its quarter
lending_ts['date'] = lending_ts['date'].dt.to_period('Q').dt.start_time

lending_ts.head()

,date,total_lending_volume
0,2011-01-01,552860801185
1,2011-04-01,557785405704
2,2011-07-01,561944383496
3,2011-10-01,567778440427
4,2012-01-01,569212229500


In [252]:
merged_df = merged_df.merge(lending_ts, on='date', how='left')

In [253]:
merged_df.head()

,date,gdp,sp500,unemployment_rate,cpi,treasury_yield,crepi,consumer_sentiment,delinquency_rate,pce,total_lending_volume
0,2011-01-01,15351.448,19.840000,NaN,NaN,NaN,NaN,73.066667,6.20,10558.200000,552860801185
1,2011-04-01,15557.539,19.506508,9.066667,224.568333,3.210000,1.867975,71.866667,5.96,10673.033333,557785405704
2,2011-07-01,15647.680,29.693906,9.000000,226.032667,2.426667,1.423464,59.666667,5.70,10755.033333,561944383496
3,2011-10-01,15842.259,31.800794,8.633333,227.047333,2.046667,6.368259,64.800000,5.46,10809.166667,567778440427
4,2012-01-01,16068.805,21.766935,8.266667,228.326000,2.036667,2.673520,75.500000,5.27,10959.300000,569212229500


In [254]:
# Calculate MEAP
merged_df['MEAP'] = (merged_df['total_lending_volume'] - merged_df['total_lending_volume'].shift(1)) / merged_df['total_lending_volume'].shift(1)

In [255]:
merged_df.head()

,date,gdp,sp500,unemployment_rate,cpi,treasury_yield,crepi,consumer_sentiment,delinquency_rate,pce,total_lending_volume,MEAP
0,2011-01-01,15351.448,19.840000,NaN,NaN,NaN,NaN,73.066667,6.20,10558.200000,552860801185,NaN
1,2011-04-01,15557.539,19.506508,9.066667,224.568333,3.210000,1.867975,71.866667,5.96,10673.033333,557785405704,0.008907
2,2011-07-01,15647.680,29.693906,9.000000,226.032667,2.426667,1.423464,59.666667,5.70,10755.033333,561944383496,0.007456
3,2011-10-01,15842.259,31.800794,8.633333,227.047333,2.046667,6.368259,64.800000,5.46,10809.166667,567778440427,0.010382
4,2012-01-01,16068.805,21.766935,8.266667,228.326000,2.036667,2.673520,75.500000,5.27,10959.300000,569212229500,0.002525


In [256]:
merged_df.to_excel("Final_MEAP.xlsx", index=False)